# Human in the Loop with Function Tools

<h3>When agents require any user input, for example to approve a function call, this is referred to as a human-in-the-loop pattern. An agent run that requires user input, will complete with a response that indicates what input is required from the user, instead of completing with a final answer. The caller of the agent is then responsible for getting the required input from the user, and passing it back to the agent as part of a new agent run.</h3>

In [1]:
# Installation refer 'basic_food_agent.ipynb' , installation section
# !pip install -U agent-framework --pre
# You can see the basic_food_agent.ipynb for basic agent run

In [2]:
# let's import the main component which will make the approval function
from typing import Annotated
from agent_framework import ai_function # ai_function can also be used to create the functiontools

import requests
import json
from typing import Annotated, List, Dict, Any, Optional
from pydantic import Field

# Clean the MealDB response for the LLM
def _clean_meal_data(meal: Dict[str, Any]) -> Dict[str, Any]:
    """
    Helper function to restructure the raw meal API response into a clean, 
    LLM-friendly format by combining ingredients and measures.
    """
    if not meal:
        return {}

    # Combine ingredients and measures into a single list
    ingredients = []
    for i in range(1, 21):
        ing = meal.get(f"strIngredient{i}")
        measure = meal.get(f"strMeasure{i}")
        if ing and ing.strip():
            ingredients.append(f"{measure.strip()} {ing.strip()}".strip())

    return {
        "id": meal.get("idMeal"),
        "name": meal.get("strMeal"),
        "category": meal.get("strCategory"),
        "area": meal.get("strArea"),
        "instructions": meal.get("strInstructions"),
        "ingredients": ingredients,
        "tags": meal.get("strTags"),
        "youtube_link": meal.get("strYoutube")
    }

# Get random meal for today, let's add the approval
@ai_function(approval_mode="always_require")
def get_random_meal() -> str:
    """
    Retrieves a random meal recipe from the database. 
    Useful when the user wants a surprise suggestion or explicitly asks for a random recommendation.

    Returns:
        str: A JSON string containing the meal name, ingredients, and cooking instructions.
    """
    try:
        response = requests.get("https://www.themealdb.com/api/json/v1/1/random.php")
        response.raise_for_status()
        data = response.json()
        
        if not data.get("meals"):
            return json.dumps({"error": "No meal found."})
            
        meal = _clean_meal_data(data["meals"][0])
        return json.dumps(meal, indent=2)
        
    except Exception as e:
        return json.dumps({"error": f"Failed to fetch random meal: {str(e)}"})

# Here we get the meal by name
def get_meal_by_name(
    meal_name: Annotated[str, Field(description="The name of the meal to search for (e.g., 'Arrabiata', 'Burger').")]
) -> str:
    """
    Searches for a specific meal recipe by name. 
    Use this when the user asks for a specific dish or wants to know how to cook a named item.

    Args:
        meal_name: The name of the dish to search for.

    Returns:
        str: A JSON string containing a list of matching meals with their details.
    """
    try:
        # The API requires a search query parameter 's'
        response = requests.get(f"https://www.themealdb.com/api/json/v1/1/search.php?s={meal_name}")
        response.raise_for_status()
        data = response.json()
        
        if not data.get("meals"):
            return json.dumps({"status": "not_found", "message": f"No meals found with the name '{meal_name}'."})
        
        # Clean and limit results (e.g., top 3 matches to save tokens)
        results = [_clean_meal_data(m) for m in data["meals"][:3]]
        return json.dumps(results, indent=2)

    except Exception as e:
        return json.dumps({"error": f"Failed to search for meal: {str(e)}"})

In [3]:
# Import core dependencies to create the agent, for Agent Framework
import asyncio
import os
import json

from dotenv import load_dotenv, find_dotenv
# Core components for building Agent, tool-enabled agents
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

In [4]:
# load environment file
load_dotenv(find_dotenv())

True

In [5]:
# Setup OpenAIChatClient for LLM Inference - Here we will use OpenRouter API which is compatible with OpenAI and NVIDIA 30B model
# This client connects to the OpenRouter Models which are OpenAI-compatible endpoint
# Environment variables required
# OPENROUTER_ENDPOINT - 
# OPENROUTER_API_KEY
openai_chat_client = OpenAIChatClient(
    base_url=os.environ.get("OPENROUTER_ENDPOINT"),
    api_key=os.environ.get("OPENROUTER_API_KEY"),
    model_id="nvidia/nemotron-3-nano-30b-a3b:free"
)

<h2> Here we configure the System Prompt for Agent with Name</h2>

In [6]:
AGENT_NAME = "FoodAgent"

AGENT_INSTRUCTIONS = """You are an expert AI Chef dedicated to helping users discover and prepare delicious meals.

CORE BEHAVIORS:
1. **Tool Usage**: You have access to a recipe database. ALWAYS use the provided tools to answer questions about recipes. Do not guess or hallucinate ingredients.
   - Use `get_meal_by_name` when the user asks for a specific dish.
   - Use `get_random_meal` when the user is undecided, asks for a suggestion, or wants a surprise.

2. **Response Format**: 
   - Start with an appetizing description of the dish.
   - List key ingredients clearly (based on the tool output).
   - Summarize the cooking instructions to be easy to follow.
   - If the tool provides a YouTube link, always include it at the end.
   - Include Tool Name used for fetching response

3. **Constraints**: 
   - Keep your response friendly but strictly under 200 words. 
   - If instructions are long, summarize the key steps to fit the word limit.
"""

In [7]:
# create the agent remember we are not using any tools here, this is simple example
agent = ChatAgent(
    name = AGENT_NAME,
    chat_client=openai_chat_client,
    instructions=AGENT_INSTRUCTIONS,
    tools=[get_random_meal]
)

In [8]:
result = await agent.run("What I can eat today?")

# here we will handle the `Human In the Loop` functionality
if result.user_input_requests:
    # Verify if the USER INPUT is needed and print accordingly
    for user_input_needed in result.user_input_requests:
        print("USER INPUT is NEEDED for BELOW FUNCTION")
        print(f"Function: {user_input_needed.function_call.name}")
        print(f"Arguments: {user_input_needed.function_call.arguments}")

USER INPUT is NEEDED for BELOW FUNCTION
Function: get_random_meal
Arguments: {}


In [9]:
from agent_framework import ChatMessage, Role

# Get user approval (in a real application, this would be interactive)
user_approval = True  # or False to reject

# Create the approval response
approval_message = ChatMessage(
    role=Role.USER, 
    contents=[user_input_needed.create_response(user_approval)]
)

# Continue the conversation with the approval
final_result = await agent.run([
    "What can I eat today?",
    ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
    approval_message
])
print(final_result.text)

**Meal Suggestion:** 🎉 **Kung Po Prawns** – a fiery, nutty Chinese seafood stir‑fry that’s quick to whip up and bursting with flavor!  

**Key Ingredients**  
- 400 g Prawns  
- 2 Tbsp Soy Sauce  
- 1 tsp Corn Flour  
- 1 tsp Tomato Puree  
- 1 tsp Caster Sugar + 1 Tbsp Brown Sugar  
- 1 tsp Sunflower Oil  
- 85 g Peanuts  
- 3 Large Chillies  
- 450 g Water Chestnuts  
- 6 Garlic Cloves, ginger & garlic  

**Quick Prep & Cook**  
1️⃣ Toss prawns with cornflour, 1 Tbsp soy sauce, and marinate 10 min.  
2️⃣ Mix vinegar, remaining soy sauce, tomato puree, sugar & water → sauce.  
3️⃣ Heat wok, fry prawns until golden, set aside.  
4️⃣ Stir‑fry peanuts, chilies, water chestnuts 2 min, add ginger & garlic 1 min.  
5️⃣ Return prawns, pour sauce, simmer 2 min until slightly thickened.  
6️⃣ Serve hot over steamed rice.  

**Watch the full video guide:** https://www.youtube.com/watch?v=ysiuZm9FIxs  

*Tool used: get_random_meal*


<h3> Now let's add the Human in the Loop with INPUT method </h3>

In [10]:
async def test_human_in_the_loop():
    query = "What I can eat today?"
    result = await agent.run(query)
    approval = None
    # here we will handle the `Human In the Loop` functionality
    if result.user_input_requests:
        # Verify if the USER INPUT is needed and print accordingly
        for user_input_needed in result.user_input_requests:
            print("USER INPUT is NEEDED for BELOW FUNCTION")
            print(f"Function: {user_input_needed.function_call.name}")
            print(f"Arguments: {user_input_needed.function_call.arguments}")
            user_approval = input("Would you like to have a random Meal ? (True/False): ")
            approval = user_approval
    new_inputs = [query]
    if approval == "True":
        # Create the approval response
        approval_message = ChatMessage(
            role=Role.USER, 
            contents=[user_input_needed.create_response(user_approval)]
        )
        new_inputs.append(approval_message)
        final_result = await agent.run(new_inputs)
        print(final_result.text)
    else:
        print("APPROVAL DENIED!!!! EXIT")
        return
    # # Continue the conversation with the approval
    # final_result = await agent.run([
    #     "What can I eat today?",
    #     ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
    #     approval_message
    # ])
    # print(final_result.text)

In [11]:
true_test = await test_human_in_the_loop()

USER INPUT is NEEDED for BELOW FUNCTION
Function: get_random_meal
Arguments: {}


Would you like to have a random Meal ? (True/False):  True


🍽️ **Cassava Pizza** – a crispy, golden flatbread made from cassava, topped with tangy tomato sauce, melty mozzarella, and a savory mix of chorizo, turkey ham, sweet corn, olives, and smoky paprika. It’s a hearty, flavorful bite that’s perfect for lunch or dinner.  

**Key Ingredients**  
- 6 thick slices Casabe (cassava flatbread)  
- 450 g tomato sauce  
- 225 g chorizo, sliced  
- 225 g turkey ham, diced  
- 75 g sweet corn  
- 40 g green olives, sliced  
- 55 g paprika  
- 50 g mozzarella, shredded  

**Quick Steps**  
1. Preheat oven to 200 °C (392 °F).  
2. Spread a thin layer of tomato sauce on each cassava slice and top with mozzarella.  
3. Layer on chorizo, turkey ham, corn, olives, and a sprinkle of paprika.  
4. Bake 7–10 minutes until the cheese bubbles and the edges turn golden.  
5. Enjoy hot straight from the oven!  

For a visual guide, check out the preparation video here: https://www.youtube.com/watch?v=3fK4zsa2AYM  

*Recipe fetched using the **get_random_meal** too

In [12]:
false_test = await test_human_in_the_loop()

USER INPUT is NEEDED for BELOW FUNCTION
Function: get_random_meal
Arguments: {}


Would you like to have a random Meal ? (True/False):  False


APPROVAL DENIED!!!! EXIT
